# Sistemas de Recomendación: Evaluación

Luego de haber visto diferentes alternativas para la implementación de los sistemas de recomendación (no personalizados, estereotipados, basados en contenido, filtrado colaborativo, factorización de matrices), es necesario preguntarse, **cuál es el mejor?**. Esta no es una pregunta sencilla dado que no hay un algoritmo que sea el mejor en todas las situaciones.

La evaluación es importante para los proyectos de machine learning, ya que permite objetivamente comparar algoritmos diferentes y sus diferentes opciones de parametrización. 

Un aspecto clave de la evaluación es garantizar que el modelo entrenado pueda generalizar sobre los datos con los que no fue entrenado. En este sentido, como ya han visto previamente, existen diferentes alternativas que pueden aplicarse. Por ejemplo, es posible utilizar validación cruzada (del inglés *cross validatio*) en el que se selecciona aleatoriamente un subconjunto de los datos para la evaluación y se utiliza el resto para el entrenamiento; repitiendo esto varias veces. En tareas con componentes temporales, otra alternativa es dividir los datos en dos conjuntos de datos de acuerdo a una fecha de referencia. Todo lo anterior a la fecha de referencia será utilizado como training y el resto como test. 

### "Ambientes" de evaluación

La evaluación puede realizarse en dos "ambientes":

* ***Offline***. Evaluación "tradicional" en la que se generalmente se tiene un dataset fijo, previamente recolectado que no ser verá modificado. Este dataset luego es separado en un único o múltiples conjuntos de training y test.
    * *Ventajas*. Más fácil de llevar a cabo. "Solo" requiere elegir un dataset entre los ya existentes. Al tener un dataset fijo (con interacciones entre los usuarios y los elementos ya fijadas), los resultados son reproducibles y es más fácil la comparación entre diferentes algoritmos.
    * *Desventajas*. La crítica más fuerte a este tipo de evaluación es su validez. El objetivo de un sistema de recomendación es proveer de nuevas reocmendaciones a os usuarios. El problema de evaluarlas en un set de test es que dse deben tener las valoraciones de los usuarios para cada item, lo que resulta en que el testing es solo realizado sobre elementos que se está seguro que el usuario ya conoce. En este sentido, si se recomeiendan items que el usuario aun no conoce, pero podrían ser una buena recomendación se penalizaría al recomendador.

* ***Online***. Realizada con usuarios reales interactuando con diferentes versiones o algoritmos de recomendación y la evaluación es realizada recolectando métricas asociadas al comportamiento del usuario en tiempo real. Por ejemplo, ver A/B testing.
    * *Ventajas*. Permite recolectar interacciones de los usuarios con los elementos recomendados en tiempo real. Además, al evaluar en tiempo real, es posible proveer más análisis.
    * *Desventajas*. Difícil, cuando no imposible, de reproducir las evaluaciones. Requiere de esfuerzo extra para configurar o crear el ambiente de las evaluaciones.  
    

Ver ([Hijikata, 2014](http://soc-research.org/wp-content/uploads/2014/11/OfflineTest4RS.pdf)) para una guía de las características de ambos tipos de evaluación.






En esta notebook nos vamos a concentrar en las métricas que pueden utilizarse en la evaluación offline.

## Métricas de evaluación
Para cada métrica vamos a ver la definición, la implementación y un ejemplo simple.

### Mean Absolute Error (MAE) - Error medio absoluto

Mide la magnitud promedio de los errores en las predicciones, sin considerar su dirección (es decir, si se trató de una sub o una sobre-estimación). Es el promedio de la diferencia absoluta entre el valor estimado y el valor real, donde todas las difererencias tienen el mismo peso.

<p align="center">
<img src="https://miro.medium.com/max/1189/1*u4kE_TFOJ1vnXDWH8ldYxw.jpeg" alt="MAE" style="width: 100px;"/>
</p>

Si no se considerase el valor absoluto, se trataría del *Mean Bias Error*, el cual puede aportar información valiosa, pero se debe tener cuidado en la interpretación dado que los errores positivos y negativos se cancelan entre sí.

No se encuentra acotada, con lo que su rango es $[0,+\infty]$. A menor valor, menor error, y entonces mejor estimación.


In [0]:
def mae(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        sum_error += abs(predicted[i] - actual[i])
    return sum_error/len(actual)

# viene implementada por defecto en las bibliotecas, por ejemplo, para usar la de surprise
def mae_(predictions):
    return accuracy.mae(predictions, verbose=False)

In [0]:
actual = [1, 2, 3, 5, 2]
predicted = [2, 2.5, 4, 4, 1]

print('MAE',mae(actual,predicted))

### Root Mean Squared Error (RMSE)

Representa la raíz cuadrada del promedio de las diferencias al cuadrado entre el valor real y el estimado. De forma similar a MAE, a menor valor, menor error. A diferencia de MAE, como las diferencias son elevadas al cuadrado, los errores más grandes tienen más peso en el valor final.

<p align="center">
<img src="https://miro.medium.com/max/655/1*DC4mfD2vplwYek--SzcEyQ.jpeg" alt="RMSE" style="width: 200px;"/>
</p>

Si bien no es una métrica acotada, se pueden definir sus límites en función del mae, donde $n$ es la cantidad de valores.

$$MAE \leq RMSE \leq MAE * \sqrt{n} $$



Tanto esta métrica como MAE no dicen nada por si solas, sino que necesitan siempre ser analizadas en comparación a otras.

In [0]:
# viene implementada por defecto en las bibliotecas, por ejemplo, para usar la de surprise
def rmse(actual, predicted):
  error = 0.0
  # TODO
  return error

def rmse_(predictions):
  return accuracy.rmse(predictions, verbose=False)

In [0]:
actual = [1, 2, 3, 5, 2]
predicted = [2, 2.5, 4, 4, 1]

print('RMSE',rmse(actual,predicted))

### Hit rate

A diferencia de las métricas anteriores, no se mide el error en los ratings, sino que se mide la calidad del ranking generado.

$$Hit Rate = \frac{\#\ hits\ in\ test}{\#\ recommended \ items} $$

Se le da más prioridad a la cantidad total de elementos correctos recomendados que a la cantidad de elementos correctos recomendados por usuario. Por ejemplo, si se tienen dos usuarios, a los cuales se le recomienda 10 elementos a cada uno, tanto como si a un usuario se le recomiendan los 10 elementos de forma correcta y al otro cero, o si a cada usuario se le recomienda correctamente cinco elementos, la métrica dará el mismo resultado.

Si son proporcionalmente pocos los elementos correctos a recomendar es muy difícil que la métrica alcance valores altos, a menos que se cuente con un gran dataset.


In [0]:
def HitRate(topNPredicted, realElements):
    hits = 0
    total = 0
    #for each left out rating
    for real in realElements:
        userID = real[0]
        leftOutMovieID = real[1]
        #is it in predicted top 10 for this user?
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            #print(movieID)
            if (leftOutMovieID == movieID):
               hit = True
               break
        if (hit):
            hits += 1
        total += 1
    #compute overall precision
    return (hits/total)

In [0]:
predicted = {}
predicted[1] = [('A',3.4),('B',2.3),('C',1)] 
predicted[2] = [('D',3.4),('E',2.3),('F',1)] 

realElements = [(1,'A',3,3.3),(2,'E',1,2),(1,'V',3,4)]

print('Hit Rate:',HitRate(predicted,realElements))


### Average Reciprocal Hit Rank

Una variación de hit rate. La diferencia es que se considera "donde" en la listas recomendadas se encuentran los items correctos. Se da más peso a la recomendación correcta de items en las posiciones más altas del ranking que en las más bajas. 

$$ ARHR = \frac{1}{\#\ elements}\sum_{i=1}^{\#hits}\frac{1}{pos_i} $$

Al igual que Hit Rate, a mayor puntaje, mejores sresultados. A diferencia de Hit Rate se encuentra más orientada a los usuarios, dado que se asume que ellos tienden a enfocarse más en los items al principio de la lista.


In [0]:
def AverageReciprocalHitRank(topNPredicted, realElements):
    summation = 0
    total = 0
    # For each real rating
    for userID, realMovieID, actualRating, estimatedRating, _ in realElements:
        # Is it in the predicted top N for this user?
        hitRank = 0
        rank = 0
        for movieID, predictedRating in topNPredicted[int(userID)]:
            rank = rank + 1
            if (realMovieID == movieID):
                hitRank = rank
                break
        if (hitRank > 0) :
           summation += 1.0 / hitRank
        total += 1

    return summation / total

In [0]:
# recuerden que no es necesario inicializarlos cada vez, están para recordar los elementos que tienen
predicted = {}
predicted[1] = [('A',3.4),('B',2.3),('C',1)] 
predicted[2] = [('D',3.4),('E',2.3),('F',1)] 

realElements = [(1,'A',3,3.3,2),(2,'E',1,2,2),(1,'V',3,4,2)]

print('Average Reciprocal Hit Rate:',AverageReciprocalHitRank(predicted,realElements))

### Cumulative Hit Rate

Otra variación del Hit Rate en la que se descartan los ratings estimados si se encuentran por debajo de un threshold. La suposición detrás de esto que no se debe "sumar puntos" por recomendar items que los usuarios no van a disfrutar. Al igual que las métricas anteriores, a mejor puntaje, mejores resultados.


In [0]:
# ratingCutoff qué rating tiene que tener para que lo considere?

def CumulativeHitRate(topNPredicted, realElements, ratingCutoff=2):
        hits = 0
        total = 0
        #for each real rating
        for userID, realMovieID, actualRating, estimatedRating, _ in realElements:
            #only look at ability to recommend things the user actually liked...
            if(actualRating >= ratingCutoff):
                #is it in predicted top 10 of this user?
                hit=False
                for movieID, predictedRating in topNPredicted[int(userID)]:
                    if(realMovieID == movieID):
                        hit = True
                        break
                if(hit):
                    hits += 1
                total += 1
        #compute overall precision
        if(total > 0):
            return (hits/total)
        return 0

In [0]:
predicted = {}
predicted[1] = [('A',3.4),('B',2.3),('C',1)] 
predicted[2] = [('D',3.4),('E',2.3),('F',1)] 

realElements = [(1,'A',3,3.3,_),(2,'E',1,2,_),(1,'V',3,4,_)]

print('Cumulative Hit Rate 0:',CumulativeHitRate(predicted,realElements))
print('Cumulative Hit Rate 1:',CumulativeHitRate(predicted,realElements,1))
print('Cumulative Hit Rate 2:',CumulativeHitRate(predicted,realElements,2))
print('Cumulative Hit Rate 3:',CumulativeHitRate(predicted,realElements,3))
print('Cumulative Hit Rate 4:',CumulativeHitRate(predicted,realElements,4))
print('Cumulative Hit Rate 5:',CumulativeHitRate(predicted,realElements,5))

### Rating Hit Rate

Otra forma de calcular el Hit Rate, separado de acuerdo a los ratings de los elementos. En lugar de mantener un única variable donde se cuentan los hits, se tiene una por cada valor posible de rating.



In [0]:
from collections import defaultdict

def RatingHitRate(topNPredicted, realElements):
    hits = defaultdict(float)
    total = defaultdict(float)

    # For each real rating
    for userID, realMovieID, actualRating, estimatedRating, _ in realElements:
        # Is it in the predicted top N for this user?
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            if (realMovieID == movieID):
                hit = True
                break
        if (hit) :
            hits[actualRating] += 1

        total[actualRating] += 1

    # Compute overall precision
    for rating in sorted(hits.keys()):
        hits[rating] = hits[rating] / total[rating]

    return hits

In [0]:
predicted = {}
predicted[1] = [('A',3.4),('B',2.3),('C',1)] 
predicted[2] = [('D',3.4),('E',2.3),('F',1)] 

realElements = [(1,'A',3,3.3,_),(2,'E',1,2,_),(1,'V',3,4,_)]

print('Rating Hit Rate:',RatingHitRate(predicted,realElements))

### Precision

La precisión se define como la cantidad de elementos relevantes sobre la cantidad total de elementos recomendados. En otras palabras, la cantidad de películas correctamente recomendadas sobre el total de recomendaciones.

<p align="center">
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/5b7d5cd5010efe2ef51e7731f2124a2156830fbe" alt="RMSE" style="width: 400px;"/>
</p>

Dos posibilidades:
1. Se asume una relevancia binaria. Es decir, por cada elemento recomendado se analiza si es o no relevante controlando si el usuario le había asignado un rating previamente. 
2. Considerando la escala de ratings, se define un *threshold* a partir del cual considerar relevante o correcta una predicción. Por ejemplo, es posible definir que todos los items con un rating mayor a 3.5 son correctamente recomendados.

De forma similar $precision@n$ define cuantos elementos fueron correctamente recomendados de entre los primeros $n$ elementos recomendados. Todo lo que se encuentre rankeado más abajo de la posición $n$ es descartado.

Esta métrica se calcular por usuario y luego se promedia para todos los usuarios.



In [0]:
  import numpy as np
  
  """
  actual : a list of lists
    Actual items to be predicted
  predicted : a list of lists
    Ordered predictions
  
  Returns: precision: float
  """

def precision(predicted, actual):

    def calc_precision(predicted, actual):
        prec = [value for value in predicted if value in actual]
        prec = np.round(float(len(prec)) / float(len(predicted)), 4)
        return prec

    precision = np.mean(list(map(calc_precision, predicted, actual)))
    return precision

In [0]:
predicted = [['X', 'Y', 'Z'], ['X', 'Y', 'Z']]
actual = [['A', 'B', 'X'], ['A', 'B', 'Y']]
print('Precision',precision(predicted,actual))

**Tarea!** Cómo implementarían la métrica para que tome los resultados en el formato que teníamos antes?
Tienen que comparar el ranking que se generó para la recomendación (el que usaba los ratings estimados), con los elementos que son relevantes para el usuario. Vamos a asumir que los elementos relevantes son aquellos que se encuentran en las primeras N posiciones del ranking hecho con los ratings reales.

Una [ayuda](https://surprise.readthedocs.io/en/stable/FAQ.html).

In [0]:
def precision(topNPredictions,realElements):
    precision = 0
    totalusers = 0
    # TODO
    return precision/totalusers

In [0]:
predicted = {}
predicted[1] = [('A',3.4),('B',2.3),('C',1)] 
predicted[2] = [('D',3.4),('E',2.3),('F',1)] 

realElements = [(1,'A',3,3.3,_),(2,'E',1,2,_),(1,'V',3,4,_)]

print('Precision',precision(predicted,realElements))

### Recall

Similar a precision, solo que en este caso se calcula la cantidad de elementos correctamente recomendados sobre la cantidad de elementos relevantes y no sobre el total de los elementos recomendados.

<p align="center">
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/43a4548e95fde15433d8e3cd3c80ced433f54abe" alt="RMSE" style="width: 400px;"/>
</p>

In [0]:
  """
  actual : a list of lists
    Actual items to be predicted
  predicted : a list of lists
    Ordered predictions
  
  Returns: recall: float
  """
def recall(predicted, actual):

    def calc_recall(predicted, actual):
        reca = [value for value in predicted if value in actual]
        reca = np.round(float(len(reca)) / float(len(actual)), 4)
        return reca

    recall = np.mean(list(map(calc_recall, predicted, actual)))
    return recall

In [0]:
predicted = [['X', 'Y', 'Z'], ['X', 'Y', 'Z']]
actual = [['A', 'B', 'X'], ['A', 'B', 'Y']]
print('Recall',recall(predicted,actual))

**Tarea!** De forma similar a cómo lo calcularon para Precisión, ahora lo necesitamos para recall...

In [0]:
predicted = {}
predicted[1] = [('A',3.4),('B',2.3),('C',1)] 
predicted[2] = [('D',3.4),('E',2.3),('F',1)] 

realElements = [(1,'A',3,3.3,2),(2,'E',1,2,2),(1,'V',3,4,2)]

print('Recall',recall(predicted,realElements))

### Normalised Discounted Cumulative Gain (nDCG)

Considerar por ejemplo una aplicación donde se obtiene una lista de artículos relacionados con un cierto tópico. Algunos artículos son relevantes, mientras que otros son neutrales y otros son malos. Si se asigna un puntaje a la relevancia de cada artículo la ganancia acumulada (*cumulative gain*) puede ser definida como:

$$CG_{p} = \sum_{i=1}^{p} rel_{i}$$

Está métrica resulta simple e intuitiva, pero no considera dos aspectos:
* Es independiente de la posición en el ranking. Cambios en la posición de los elementos en el ranking no afecta al puntaje final. 
* Es dependiente de la escala. Listas más grandes tenderán a tener puntajes mayres.

En este contexto, para resolver el primer problema, se define el *discounted cumulative gain* donde la relevancia de cada elemento disminuye a medida que se encuentra más abajo en el ranking. La importancia de cada posición puede disminuir de forma lineal o incluso de forma logarítmica:

$$DCG_{p} = \sum_{i=1}^{p} \frac{rel_{i}}{log_{2}(i+1)}$$

Luego, para resolver el segundo problema, se define el *normalized $DCG$*, que es "solo" dividir el $DCG$ por un factor de normalización. Este factor es el "$DCG$ ideal", es decir el mejor $DCG$ que se podría alcanzar considerando el caso en el que todos los elementos relevantes se ubiquen en las primeras posiciones del ranking. 

Tomando un ejemplo de [Wikipedia](https://en.wikipedia.org/wiki/Discounted_cumulative_gain). Tenemos elementos que cuentan con 4 niveles de relevancia: 0, 1, 2 y 3. A mayor nivel, más relevancia. Luego, se le realiza una recomendación a un usuario donde se le retorna la siguiente secuencia de niveles de relevancia: $[3, 2, 3, 0, 1, 2]$. En este cotexto, la *cumulative gain* será:

$$CG_{p} = \sum_{i=1}^{p} rel_{i} = 3 + 2 + 3 + 0 + 1 + 2 = 11$$

Es importante destacar que si se altera el orden de los documentos, el resultado va a ser el mismo. Por otra parte, el valor de $DCG$ será:

$$DCG_{p} = \sum_{i=1}^{p} \frac{rel_{i}}{log_{2}(i+1)} = 3 + 1.262 + 1.5 + 0 + 0.387 + 0.712 = 6.861$$

Finalmente, para calcular el $nDCG$, primero se crea el ranking ideal, ordendando a los items de acuerdo a su relevancia y luego se calcula el $DCG$ de ese raking.


|Original Rank| Ideal Rank| OriginalDCG| IdealDCG | $\frac{DCG}{IdealDCG}$|
|---|---|---|---|---|
|[3, 2, 3, 0, 1, 2]| [3, 3, 2, 2, 1, 0] | 6.861 | 8.740| 0.785|

**Nota.** Si bien en el ejemplo se trabajó con relevancia numérica, es posible también considerar relevancia binaria, en la que se asiga una reevancia de $1$ a los elementos efectivamente relevantes, y una de $0$ a los irrelevantes.
 

In [0]:
import math

def idcg(predicted):
    itemRelevance = 1
    idcg_ = 0
    i = 1
    for x in range(0,len(predicted)):
        idcg_ += (itemRelevance) / math.log(i+1,2)
    i += 1
    return idcg_

def ndcg(predicted, realElements, user):

    idcg_m = idcg(predicted)
    itemRelevance = 1
    i = 1
    dcg = 0
    real_movies = [ x[1] for x in realElements]

    for movieID, predictedRating in predicted[user]:
        if movieID in real_movies:
            dcg += itemRelevance / math.log(i+1,2)
        i += 1
    print(dcg)
    return dcg / idcg_m

In [0]:
predicted = {}
predicted[1] = [('A',3.4),('B',2.3),('C',1)] 

realElements = [(1,'A',3,3.3,2),(1,'V',3,4,2)]

print('NDCG',ndcg(predicted,realElements,1))

Ojo! El $ndcg$ así como está, calcula la métrica para un único usuario. Cómo harían para calcular el ndcg promedio para todos los usuarios. 

Pista. Tienen que encontrar la forma de seleccionar sub conjuntos de realElements. Uno para cada usuario.

In [0]:
# TODO: ndcg promedio para todos los usuarios

---

Hasta acá venimos midiendo la calidad de las recomendaciones de una forma casi estadística. Sin embargo, no resultan los únicos aspectos a considerar. También es posible considerar métricas que se encuentran más relacionadas con la lógica del negocio o con la experiencia al usuario.

<p align="center">
<img src="https://miro.medium.com/max/780/1*5L_T_-yH1yr-aEX5_tcPNA.png" alt="Evaluation metrics" style="width: 400px;"/>
</p>



### Coverage

Existen diferentes definiciones.
1. Qué porcentaje de los items totales en el sistema son efectivamente recomendados? (es decir, la cantidad de elementos distintos recomendados divididos la cantidad de items distintos a recomendar)
2. Qué porcentaje de usuarios recibió al menos una recomendación correcta o relevante? Al igual que en los casos anteriores, la relevancia puede ser medida de forma binaria o con un threhsold.

Esta métrica presenta un *trade-off* con precición, dado que los sistemas de recomendación pueden obtener un algo coverage a expensas de una precisión baja, mientras que sistemas con alta precisión por lo general tendrán un coverage bajo.




In [0]:
def userCoverage(topNPredicted, numUsers, ratingThreshold=0):
    hits = 0
    for userID in topNPredicted.keys():
        hit = False
        for movieID, predictedRating in topNPredicted[userID]:
            if (predictedRating >= ratingThreshold):
                hit = True
                break
        if (hit):
           hits += 1

    return hits / numUsers

In [0]:
predicted = {}
predicted[1] = [('A',3.4),('B',2),('C',1)] 
predicted[2] = [('D',4.4),('E',2.3),('F',1)] 

print('UserCoverage',userCoverage(predicted,2))

### Diversity

Una opción sencilla en recomendación es recomendar a todos los usuarios los $n$ elementos más populares. Sin embargo, esto puede no ser la mejor opción, para qué querríamos un sistema de recoemndación que nos recomiende lo que probablemente ya conozcamos? Asimismo, de qué nos sirve que a todos nos recomiende lo mismo o elementos del mismo tipo? 

En este contexto, esta métrica mide cuan diferentes son los elementos que se le recomiendan al usuario. Por lo general se mide utilizando los metadatos de los elementos, por ejemplo, la categoría, género, tags, ... Por ejemplo, cuántas categorías diferentes de elementos fueron recomendadas? Si se cuenta con las recomendaciones a múltiples usuarios, es posible calcular la semejanza entre los elementos y optimizar para valores de semejanza bajos. Finalmente, si se cuenta con valores de ratings, es posible considerar la diversidad como la dispersión de los ratings en la lista de recomendación. A mayor desvío standard, mayor diversidad de la lista.


In [0]:
# para esta métrica no solo se necesita conocer qué elementos se recomendaron, sino también la semejanza entre todos los elementos
def diversity(topNPredicted, simsAlgo):
    n = 0
    total = 0
    simsMatrix = simsAlgo.compute_similarities()
    for userID in topNPredicted.keys():
        pairs = itertools.combinations(topNPredicted[userID], 2)
        for pair in pairs:
            movie1 = pair[0][0]
            movie2 = pair[1][0]
            innerID1 = simsAlgo.trainset.to_inner_iid(str(movie1)) # como surprise mantiene ids distintos, es necesario recuperarlos
            innerID2 = simsAlgo.trainset.to_inner_iid(str(movie2))
            similarity = simsMatrix[innerID1][innerID2]
            total += similarity
            n += 1

    S = total / n
    return (1-S)

### Serendipity

Como mencionamos anteriormente, uno de los problemas de Item-Item collaborative filtering es la dificultad para proveer recomendaciones innovativas de elementos que son conocidos por pocos usuarios (es decir, no son populares), pero que podrían ser una excelente recomendación. 

Otro aspecto relevante de la serendipia, es la evolución temporar de los intereses de los usuarios. Por ejemplo, [Neal Lathia](https://www.coursera.org/learn/recommender-metrics/lecture/twHNp/temporal-evaluation-of-recommenders-interview-with-neal-lathia) estudió como la satisacción de los usuarios se modificaba a medida que recibían una y otra vez la misma recomendación a lo largo del tiempo. El estudio mostró que cuando los usuarios reciben buenas recomendaciones, si sus intereses no cambian, su satisfacción se ve disminuída a lo largo del tiempo.

Una posible definición de serendia es la propuesta por  [Murakami, 2008](https://www.researchgate.net/publication/225121950_Metrics_for_Evaluating_the_Serendipity_of_Recommendation_Lists):

$$ser_{mur}(u) = \frac{1}{|R_{u}|}\sum_{i\in R_{u}}max(Pr_{u}(i) - Prim_{u}(i),0) * rel_{u}(i)$$

donde $Pr_{u}(i) - Prim_{u}(i)$ mide la serendipia de cada item en el conjunto de todas las posibles recomendaciones. Contiene el rating esperado por un nuevo sistema de recomendación, respecto al rating provisto por un sistema de recomendación tradicional. Valores cercanos a cero, son reemplazados por cero para evitar realizar recomendaciones que no sean de agrado para los usuarios. Luego, la diferencia es multiplicada por la relevancia de los items, dado que solo se quieren recomendar elementos que ya sabemos (o creemos) que le van a gustar al usuario. Finalmente, estas diferencias son promedidas para tener una idea de cuanta "sorpresa" es capaz de proveer el sistema de recomendación.


Para una discusión acerca de los desafíos para la definición y cálculo de serendipia, referirse a ([Kotkov, 2016](https://www.sciencedirect.com/science/article/pii/S0950705116302763)).





### Personalization

Se trata de una buena forma de analizar si se estan recomendando items similares a los diferentes usuarios. Se define como la "dissimilarity" $1 - cosine similarity$) entre las listas de los diferentes usuarios. 

Un alto valor de personalización indica que las recomendaciones a los usuarios son diferentes, lo que significa que el recomendador está ofrenciendo recomendaciones efectivamente personalizadas a las características e interess de los usuarios.


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

def personalization(predicted):

    #create matrix for recommendations
    
    #calculate similarity for every user's recommendation list
  
    #get indicies for upper right triangle w/o diagonal
    upper_right = np.triu_indices(similarity.shape[0], k=1)

    #calculate average similarity
    personalization = np.mean(similarity[upper_right])

    return 1-personalization


In [0]:
predicted = [['X', 'Y', 'Z'], ['X', 'Y', 'Z']]
print("Personalization:",personalization(predicted))

predicted = [['X', 'A', 'B'], ['X', 'Y', 'Z']]
print("Personalization:",personalization(predicted))

predicted = [['A', 'B', 'C'], ['X', 'Y', 'Z']]
print("Personalization:",personalization(predicted))


---

Las métricas presentadas se encuentran implementadas en diferentes bibliotecas disponibles públicamente: [recmetrics](https://github.com/statisticianinstilettos/recmetrics) y [recommendermetrics](https://www.kaggle.com/l0new0lf/recommendermetrics), entre otras opciones.



## Comparando algoritmos

Al igual que en las notebooks anteriores, vamos a trabajar con el dataset de películas y ratings. Asimismo, no solo usaremos los algoritmos implementados por nosotros, sino que también utilizaremos una biblioteca. Nos basaremos en [SurPRiSE (Simple Python RecommendatIon System Engine)](http://surpriselib.com/). Esta biblioteca:
* Ofrece un fuerte énfasis en la documentación.
* Incluye datasets pre-cargados y la posibilidad de cargar datasets propios.
* Provee algoritmos baseline, vecinos, factorización de matrices.
* Provee diversas métricas de semejanza.
* Ofrece flexibilidad para la implementación de nuevos algoritmos.
* Provee herramientas para evaluar, analizar y comparar la performance de algoritmos.

In [0]:
import pandas as pd
import numpy as np

# instalamos surprise...
!pip install scikit-surprise
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import train_test_split

In [0]:
# cargamos el dataset
url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/ml-100k/u.data'
df = pd.read_csv(url, sep='\t', names=['user_id','movie_id','rating','timestamp'])

url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/ml-100k/u.item'
movie_info = pd.read_csv(url,sep='|', encoding='latin-1', header=None, names=['movie_id','movie_title','release_date','movie_release_date',
                                                                              'IMDb url','unknown','Action','Adventure','Animation','Children','Comedy',
                                                                              'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical',
                                                                              'Mystery','Romance','Sci-Fi','Thriller','War','Western'])

movie_info = movie_info.drop('movie_release_date', axis=1) # eliminamos la columna movie_release_date que es NaN para todos los registros
movie_info = movie_info.drop('IMDb url', axis=1) # eliminamos esta columna que no aporta ninguna información relevante

df = pd.merge(df, movie_info, on='movie_id')
df.head()

Una vez que cargamos el dataset, vamos a acomodarlo para poder utilizarlo con surprise

In [0]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'movie_title', 'rating']], reader)
print(data)

Como dijimos previamente, para poder evaluar la calidad de las recomendaciones debemos definir las particiones de datos que se utilizarán para el training y test de los algoritmos. 

Surprise provee varias [alternativas](https://surprise.readthedocs.io/en/stable/model_selection.html). En este caso vamos a usar la partición sencilla en un único conjunto de training conteniendo el $75\%$ de los ratings y un conjunto de test con el $25\%$ restante.

In [0]:
trainset, testset = train_test_split(data, test_size=0.25)
print(trainset)

### Evaluación de los ratings estimados

Luego, seleccionamos el algoritmo a utilizar para, en este caso, predecir los ratings de los usuarios. 
[Acá](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html) hay una descripción de los diferentes algoritmos que se encuentran disponibles. Para estas pruebas, vamos a utilizar factorización de matrices, mediante SVD.

In [0]:
algo = SVD()
# entrenamos el modelo con el training set
algo.fit(trainset)

Una vez entrenado el modelo, podemos evaluarlo sobre el test set que habíamos separado.

In [0]:
predictions = algo.test(testset)

Luego, creamos un `data frame` para visualizar los resultados.

In [0]:
test = pd.DataFrame(predictions)
test.drop("details", inplace=True, axis=1)
test.columns = ['userId', 'movieId', 'actual', 'cf_predictions']
test.head()

Cómo estamos comparando la estimación de los ratings con los ratings originalmente asignados por los usuarios, vamos a utilizar métricas de error de estimación.

In [0]:
# mean absolute error
mae = accuracy.mae(predictions, verbose=False)
print("MAE: ",mae)

# root square mean error
rmse = accuracy.rmse(predictions, verbose=False)
print("RMSE: ",rmse)

Vamos a seleccionar otro algoritmo para compararlo con el que SVD. En este caso, vamos a usar KNN.

In [0]:
# importo el nuevo algoritmo
from surprise import KNNBasic

algo = KNNBasic()
# entrenamos el modelo con el training set
algo.fit(trainset)
predictions_knn = algo.test(testset)

# mean absolute error
mae = accuracy.mae(predictions_knn, verbose=False)
print("MAE: ",mae)

# root square mean error
rmse = accuracy.rmse(predictions_knn, verbose=False)
print("RMSE: ",rmse)

**Tarea!** Elijan un algoritmo provistos por surprise, entrenarlo y evaluarlo. Cómo se comporta en comparación a los otros?

In [0]:
# importamos el nuevo algoritmo

algo = # TODO

# entrenamos el modelo con el training set
algo.fit(trainset)
predictions_new = algo.test(testset)

# mean absolute error
mae = accuracy.mae(predictions_knn, verbose=False)
print("MAE: ",mae)

# root square mean error
rmse = accuracy.rmse(predictions_knn, verbose=False)
print("RMSE: ",rmse)

Vamos a comparar algunos más...

In [0]:
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import BaselineOnly
from surprise import SVDpp #muuy lento
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

benchmark = []
# iteramos sobre los algoritmos...
for algorithm in [SVD(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # en lugar de hacer training y test, en este caso vamos a usar cross validation
    print(algorithm)
    results = cross_validate(algorithm, data, measures=['MAE','RMSE'], cv=3, verbose=False)
    
    # obtenemos los resultados y los "juntamos todos" en benchmark
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

In [0]:
benchmark

In [0]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results

### Evaluación de recomendaciones

Ahora, vamos a cambiar el foco de la evaluación. En lugar de evaluar las estimaciones de los rankings, vamos a evaluar y comparar las recomendaciones realizadas. Cómo? Vamos a utilizar las estimaciones de los rankings para hacer las predicciones.


In [0]:
from collections import defaultdict

#GetTopN takes in complete list of ratings prediction that come back from some recommender and
    #returns a dictionary that maps user ids to their Top N Ratings.
    #We are using defaultdict object which is simmilar to normal python dictionary  but has 
    #concept of default empty values
def GetTopN(predictions, n=10, minimumRating=1.0):
    topN = defaultdict(list)
    for userID, movieTitle, actualRating, estimatedRating, _ in predictions:
        if (estimatedRating >= minimumRating):
            topN[int(userID)].append((movieTitle, estimatedRating))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userID)] = ratings[:n]

    return topN

Ahora vamos a utilizar otro tipo de modelo, el `LeaveOneOut`. Es una forma de cross validation en la que cada usuario tiene exactamente 1 elemento en el test set. A diferencia de otras estrategias de cross validation, no se garantiza que todos los folds que se generen sean diferentes, aunque con datasets grandes es altamente probable.

In [0]:
from surprise.model_selection import LeaveOneOut

algo = SVD() # volvemos a setear el SVD como algoritmo
LOOCV = LeaveOneOut(n_splits=1,random_state=1)

for trainSet, testSet in LOOCV.split(data):
    # Train model without left-out ratings
    algo.fit(trainSet)
    # Predicts ratings for left-out ratings only
    leave_one_out_predictions = algo.test(testSet)
    # Build predictions for all ratings not in the training set
    bigTestSet = trainSet.build_anti_testset()
    allPredictions = algo.test(bigTestSet)
    # Compute top 10 recs for each user
    topNPredicted = GetTopN(allPredictions, n=10)
    

In [0]:
topNPredicted

In [0]:
def compute_metrics(topNPredicted, predictions):
  metrics = {}
  metrics['Hit Rate'] = HitRate(topNPredicted, predictions)
  metrics['Cumulative Hit Rate'] = CumulativeHitRate(topNPredicted, predictions, 4.0)
  metrics['Average Reciprocal Hit Rank'] = AverageReciprocalHitRank(topNPredicted, predictions)

  #metric['Precision'] = #TODO
  #metric['Recall'] = # TODO
  #metric['NDCG'] = # TODO

  return metrics

compute_metrics(topNPredicted,leave_one_out_predictions)

**Tarea!** Considerar un modelo de entrenamiento de K-Folds, donde $K=5$. Elegir el algoritmo que quieran de los que probamos anteriormente y:
* Encontrar el mejor modelo (de los modelos creados para cada uno de los folds, cuál es el que mejores resultados alcanza?). Tener en cuenta no solo el ranking generado, sino también la estimación de los ratings (son métricas diferentes!)
* Calcular el promedio de las métricas para los K folds evaluados.

In [0]:
from surprise.model_selection import KFold

kf = KFold(n_splits=5)